In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.set_option('display.max_columns', 250)

In [ ]:
API_KEY = ''

# Ranker Stats 추출

## Ranker Name 추출

In [ ]:
#api 호출로 랭커의 기본정보 불러오기
#기본정보에서 랭커의 캐릭터 이름만 추출
headers = {
    'accept': 'application/json',
    'x-nxopen-api-key': API_KEY,
}

params = {
    'date': '',#날짜 넣기
    'world_name': '',#월드 이름 넣기
}

response_name = requests.get('https://open.api.nexon.com/maplestory/v1/ranking/overall', params=params, headers=headers)

In [ ]:
ranker_name_json = response_name.json()
print(ranker_name_json['ranking'][0])

In [ ]:
#랭커 캐릭터들의 이름만 추출하여 리스트 작성
Ranker_name_list = []
for data in ranker_name_json['ranking']:
    Ranker_name_list.append(data['character_name'])
Ranker_name_list

In [ ]:
Ranker_name_list_1 = Ranker_name_list[:157]
Ranker_name_list_2 = Ranker_name_list[157:]

In [ ]:
print(len(Ranker_name_list_1), len(Ranker_name_list_2))

## Ranker Name을 ocid로 변환

In [ ]:
def name_to_ocid(name_list):
    ocid_list = []

    for name in name_list:
        headers = {
        'accept': 'application/json',
        'x-nxopen-api-key': API_KEY,
        }

        params = {
        'character_name': name,
        }

        response_ocid = requests.get('https://open.api.nexon.com/maplestory/v1/id', params=params, headers=headers)
        ocid_list.append(response_ocid.json().values())
    return ocid_list

In [ ]:
ocid_list_1 = name_to_ocid(Ranker_name_list_1)

In [ ]:
ocid_list_2 = name_to_ocid(Ranker_name_list_2)

## ocid를 이용하여 Character의 Stats 불러오기

In [ ]:
#for문은 내부에 넣지 않고 외부에 사용하기
#코드 재사용하기 용이

def Load_Stats(ocid):
    
    headers = {
        'accept': 'application/json',
        'x-nxopen-api-key': API_KEY,
    }

    params = {
        'ocid': ocid,
    }

    response_basic = requests.get('https://open.api.nexon.com/maplestory/v1/character/basic', params=params, headers=headers)
    response_main_stats = requests.get('https://open.api.nexon.com/maplestory/v1/character/stat', params=params, headers=headers)
    response_hyper_stats = requests.get('https://open.api.nexon.com/maplestory/v1/character/hyper-stat', params=params, headers=headers)
    response_item_equipment = requests.get('https://open.api.nexon.com/maplestory/v1/character/item-equipment', params=params, headers=headers)
    response_symbol_equipment = requests.get('https://open.api.nexon.com/maplestory/v1/character/symbol-equipment', params=params, headers=headers)
    
    basic_stats = response_basic.json()
    main_stats = response_main_stats.json()
    hyper_stats = response_hyper_stats.json()
    item_equipment = response_item_equipment.json()
    symbol_equipment = response_symbol_equipment.json()
    
    row0 = {
            'character_name': basic_stats['character_name'],
            'character_gender': basic_stats['character_gender'],
            'character_class': basic_stats['character_class'],
            'character_level': basic_stats['character_level'],
           }
    
    list_main_stats = main_stats['final_stat']
    row1 = {data['stat_name']: data['stat_value'] for data in list_main_stats}
    row1['character_class'] = main_stats['character_class']
    
    list_hyper_stats = hyper_stats['hyper_stat_preset_1']
    row2 = {'Hyper ' + data['stat_type']: data['stat_level'] for data in list_hyper_stats}
    
    list_item_equipment = item_equipment['item_equipment']
    row3 ={}
    for data in list_item_equipment:
        row3.update({'Item_Equipment ' + data['item_equipment_slot']: data['item_name'],
                     'Item_Equipment ' + data['item_equipment_slot'] + ' Starforce': data['starforce']})
        
    list_symbol_equipment = symbol_equipment['symbol']
    row4 = {data['symbol_name']: data['symbol_level'] for data in list_symbol_equipment}
    
    
    row0.update(row1)
    row0.update(row2)
    row0.update(row3)
    row0.update(row4)

    return row0

In [ ]:
#api 호출 횟수 제한으로 나눠서 호출
Character_stats_list_1 = []

for ocid in ocid_list_1:
    Character_stats_list_1.append(Load_Stats(ocid))

In [ ]:
Character_stats_list_2 = []

for ocid in ocid_list_2:
    Character_stats_list_2.append(Load_Stats(ocid))

In [ ]:
len(Character_stats_list_1)

In [ ]:
len(Character_stats_list_2)

In [ ]:
df = pd.DataFrame(Character_stats_list_1)

In [ ]:
df1 = pd.DataFrame(Character_stats_list_2)

In [ ]:
#임시 저장
df.to_csv('maple_data1.csv')

In [ ]:
#임시 저장
df1.to_csv('maple_data2.csv')

In [ ]:
df1 = pd.read_csv('./maple_data1.csv', index_col = 0)
df2 = pd.read_csv('./maple_data2.csv', index_col = 0)

In [ ]:
#나눠진 데이터 합치기
df = pd.concat([df1, df2], ignore_index = True)

In [ ]:
df.shape

## 데이터 전처리

In [ ]:
#null 확인
df[df.isnull().any(axis=1)]

In [ ]:
len(df[df.isnull().any(axis=1)])

In [ ]:
#column을 type 별로 분류
numerical_list = []
categorical_list = []
for col_name in df.columns:
    try:
        if int(df.iloc[0][col_name]):
            numerical_list.append(col_name)
    except:
        categorical_list.append(col_name)

In [ ]:
print(numerical_list)

In [ ]:
print(categorical_list)

In [ ]:
df = df.reset_index()


In [ ]:
df_dropna = df.dropna()
df_dropna[df_dropna.notnull().any(axis=1)]

In [ ]:
#null제거 후 포맷을 int로 변경
for col_name in numerical_list:
    df_dropna[col_name] = df_dropna[col_name].apply(lambda x:  int(x))

In [ ]:
df_dropna.info(True)

In [ ]:
print("중복된 항목 수: ", len(df_dropna[df_dropna.duplicated()]))

In [ ]:
#전처리된 데이터 저장
df_dropna.to_csv('maple_dropna.csv')

# 데이터 불러오기

In [ ]:
df = pd.read_csv('./maple_dropna.csv')

In [ ]:
df.head()